# Final Project

### Student: Jan Carbonell

### Lab Objective:
The Objective of this project are the following: 


In [1]:
# Import comet_ml in the top of the file for experiment tracking
from comet_ml import Experiment

experiment = Experiment(api_key="WgXEAqBycAS6nrjJC5zkNTLA2",
                        project_name="general", workspace="jcllobet")


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jcllobet/general/382f158355754599b4c5922d9e60737c



In [2]:
#intial set of imports
import nltk

from nltk.corpus import wordnet, stopwords
from nltk.metrics import jaccard_distance, edit_distance
from nltk import pos_tag
from nltk.parse import CoreNLPParser
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tag import PerceptronTagger
from nltk.wsd import lesk

tagger = CoreNLPParser(url='http://localhost:9000', tagtype='ner')

import pandas as pd
import string
import regex

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

random_state = 42

#variable initialization and instantiation
tests = []
tests_lem = []
gold_std_train = []
gold_std_test = []
lem1 = []
lem2 = []


Using TensorFlow backend.


## 1. Read all pairs of sentences of the train and test set
We proceed to open and read the input file

In [3]:
def text_to_sentences(filename):
    sentence_pair_array = []
    for line in open(filename, encoding="UTF8").readlines():
        sentence_pair_array.append([s.strip() for s in line.split("\t")])
    return sentence_pair_array

# TRIAL TESTING
trial_input = text_to_sentences('./00_data/trial/STS.input_fixed.txt')
trial_classes = open('./00_data/trial/STS_fixed.gs.txt', encoding="utf-8-sig").readlines()

# TRAINING PHASE
train_input = text_to_sentences('./00_data/train/STS.input.MSRpar_vid_SMT.txt')
train_classes = open('./00_data/train/STS.gs.MSRpar_vid_SMT.txt', encoding="utf-8-sig").readlines()

# TESTING PHASE
test_input = text_to_sentences('./00_data/test-gold/STS.input.ALL.txt')
test_classes = open('./00_data/test-gold/STS.gs.ALL.txt', encoding="UTF8").readlines()

We implement the lematizer. 

## 2. Noisy entities removal functions
### Stopwords, URL's, Punctuations

In [4]:
# Removing the punctuation and lowering the case of a string
def preprocessing(line):
    
    line = line.lower()
    
    # Clean the text
    line = regex.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", line)
    line = regex.sub(r"what's", "what is ", line)
    line = regex.sub(r"\'s", " ", line)
    line = regex.sub(r"\'ve", " have ", line)
    line = regex.sub(r"can't", "cannot ", line)
    line = regex.sub(r"n't", " not ", line)
    line = regex.sub(r"i'm", "i am ", line)
    line = regex.sub(r"\'re", " are ", line)
    line = regex.sub(r"\'d", " would ", line)
    line = regex.sub(r"\'ll", " will ", line)
    line = regex.sub(r",", " ", line)
    line = regex.sub(r"\.", " ", line)
    line = regex.sub(r"!", " ! ", line)
    line = regex.sub(r"\/", " ", line)
    line = regex.sub(r"\^", " ^ ", line)
    line = regex.sub(r"\+", " + ", line)
    line = regex.sub(r"\-", " - ", line)
    line = regex.sub(r"\=", " = ", line)
    line = regex.sub(r"'", " ", line) #careful, it used to be " ". Testing again for accuracy purposes. 
    line = regex.sub(r"(\d+)(k)", r"\g<1>000", line)
    line = regex.sub(r":", " : ", line)
    line = regex.sub(r" e g ", " eg ", line)
    line = regex.sub(r" b g ", " bg ", line)
    line = regex.sub(r" u s ", " american ", line)
    line = regex.sub(r"\0s", "0", line)
    line = regex.sub(r" 9 11 ", "911", line)
    line = regex.sub(r"e - mail", "email", line)
    line = regex.sub(r"j k", "jk", line)
    line = regex.sub(r"\s{2,}", " ", line)

    #only accept alphanum
    # [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    #remove punctuation
    return line.translate(str.maketrans('', '', string.punctuation))

## 3. Word normalization
### Tokenization, Lemmatization and stemming

In [5]:
#initializing stemmer
stemmer = PorterStemmer()

#testing that it works
print(stemmer.stem("running"))
print(stemmer.stem("a"))

run
a


In [6]:
# extract the words from the sentence minus stopwords
def words_from_sent(sent):
    # tokenized
    tokenized = nltk.word_tokenize(sent)
    # remove stopwords and return
    return [word for word in tokenized if word not in nltk.corpus.stopwords.words('english')]

def stopwords_from_sent(sent):
    # tokenized
    tokenized = nltk.word_tokenize(sent)
    # remove stopwords and return
    return [word for word in tokenized if word in nltk.corpus.stopwords.words('english')]
    
# convert words to tokens
def pos_tag_from_words(words):
    return pos_tag(words)

# Function to get wordnet pos code
def wordnet_pos_code(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
# Function to get wordnet pos code
def treebank_pos_code(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('A'):
        return wordnet.ADV
    else:
        return None

# Tokens to lemmas using wordnet lemmatizer    
def tokens_to_lemmas(tokens):
    return list(map(pos_tag_to_lemmas, tokens))

def pos_tag_to_lemmas(token):    
    pos = wordnet_pos_code(token[1])
    if pos:
        return WordNetLemmatizer().lemmatize(token[0], pos=pos)
    return token[0]

def tokens_to_stemming(tokens):
    return list(stemmer.stem(token) for token in tokens)
    #print(tokens_to_stemms(['a','running','verbose','singing']))

## 4. Synset, Nammed Entity and Content Parsing


In [7]:
def lesking_sentence(pos_tagged_sentence):
    """
    Returns a sentence as the given sentece using lesker algorithms.
    The input sentence must be a pos_tagged sentence (e.g. [('The', 'DN'),
    ('sun', 'NN')]).
    """
    sentence = [i[0] for i in pos_tagged_sentence]
    result = []
    
    none_type_objects = []
    for word, tag in pos_tagged_sentence:
        # 'NoneType' object has no attribute 'name'
        try:
            result.append(lesk(sentence,word, wordnet_pos_code(tag)).name())
        except:
            result.append(word)            
    return result

def get_stanford_named_entity_chunked(sentence):
    """Given the passed a tokenized sentence, returns an array with the chunks (words and named entities) it contains, using Stanford NLP"""
    
    tokenized_s = nltk.word_tokenize(sentence)
    
    # tag and run as a normal word or a named entity (e.g. a person or an organization)
    tagged_s = tagger.tag(tokenized_s)
    #print(tagged_s)
    
    chunked_sentence = []
    last_token = ''
    last_tag = ''
    
    for tagged_token in tagged_s:
        
        token = tagged_token[0]
        tag = tagged_token[1]
        
        # make normal words have lower case, also discard punctuation marks
        if tag == 'O':
            if token.isalnum():
                chunked_sentence.append(token.lower())
         
        # keep named entities with the original capitalization
        else:
            if last_tag == tag:
                chunked_sentence[-1] += ' ' + token
            else:
                chunked_sentence.append(token)
        
        last_token = token
        last_tag = tag
    
    return chunked_sentence

# example: note it does not group the terms of named entities, always 1 by 1. 

## 6. Distances and evaluation

In [8]:
# evaluation imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
%matplotlib inline

#jaccard and edit distances
def jaccard_d(list_0, list_1):
    if len(set(list_0)) == 0 or len(set(list_1)) == 0:
        return len(set(list_0).union(set(list_1)))
    else:
        return float(jaccard_distance(set(list_0),set(list_1)))
    
def edit_d(list_0, list_1):
    return float(edit_distance(list_0,list_1))


## Main Program

In [9]:
def main_feature_generation(sent_0, sent_1):
    #print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    #print(sent_0 + '\n' + sent_1 + '\n')
    stop_0, stop_1 = stopwords_from_sent(sent_0), stopwords_from_sent(sent_1)
    token_0, token_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    #ner_0, ner_1 = get_stanford_named_entity_chunked(sent_0), get_stanford_named_entity_chunked(sent_1)
    pos_0, pos_1 = pos_tag_from_words(token_0), pos_tag_from_words(token_1)
    stems_0, stems_1 = tokens_to_stemming(token_0), tokens_to_stemming(token_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(pos_0), tokens_to_lemmas(pos_1)
    lesk_0, lesk_1 = lesking_sentence(pos_0), lesking_sentence(pos_1)
    

    featureset = {
        "stops_jaccard":jaccard_d(stop_0, stop_1),
        "stops_edit":edit_d(stop_0, stop_1),
        "tokens_jaccard":jaccard_d(token_0, token_1),
        "tokens_edit":edit_d(token_0, token_1),
        #"ner_jaccard":jaccard_d(ner_0, ner_1),
        #"ner_edit":edit_d(ner_0, ner_1),
        "pos_jaccard":jaccard_d(pos_0, pos_1),
        "pos_edit":edit_d(pos_0, pos_1),
        "stems_jaccard":jaccard_d(stems_0, stems_1),
        "stems_edit":edit_d(stems_0, stems_1),
        "lemmas_jaccard":jaccard_d(lemmas_0, lemmas_1),
        "lemmas_edit":edit_d(lemmas_0, lemmas_1),
        "lesk_jaccard":jaccard_d(lesk_0, lesk_1),
        "lesk_edit":edit_d(lesk_0, lesk_1)
    }
    
    #"NER":[] --> DO ZALANDO's IMPLEMENTATION https://github.com/zalandoresearch/flair
   
    
    return featureset


In [10]:
print('Training data')
X_train = [main_feature_generation(data[0], data[1]) for data in train_input]
df_X_train = pd.DataFrame(X_train)
training_scores_y = [float(line.strip()) for line in train_classes]
print('Finished Training!\n')
df_X_train.head()

Training data
Finished Training!



,lemmas_edit,lemmas_jaccard,lesk_edit,lesk_jaccard,pos_edit,pos_jaccard,stems_edit,stems_jaccard,stops_edit,stops_jaccard,tokens_edit,tokens_jaccard
0,7.0,0.526316,7.0,0.526316,7.0,0.526316,7.0,0.526316,5.0,0.363636,7.0,0.526316
1,5.0,0.500000,5.0,0.500000,5.0,0.500000,5.0,0.500000,3.0,0.800000,5.0,0.500000
2,7.0,0.642857,7.0,0.733333,7.0,0.642857,7.0,0.642857,4.0,0.666667,7.0,0.642857
3,6.0,0.388889,6.0,0.388889,6.0,0.388889,6.0,0.388889,3.0,0.333333,6.0,0.388889
4,14.0,0.888889,15.0,0.928571,16.0,0.965517,14.0,0.888889,4.0,0.666667,14.0,0.888889


In [11]:
print('Testing data')
X_test = [main_feature_generation(data[0], data[1]) for data in test_input]
df_X_test = pd.DataFrame(X_test)
testing_scores_y = [float(line.strip()) for line in test_classes]
print('Finished Testing!\n')

df_X_test.head()

Testing data
Finished Testing!



,lemmas_edit,lemmas_jaccard,lesk_edit,lesk_jaccard,pos_edit,pos_jaccard,stems_edit,stems_jaccard,stops_edit,stops_jaccard,tokens_edit,tokens_jaccard
0,6.0,0.714286,8.0,0.941176,8.0,0.875000,5.0,0.615385,5.0,0.625000,7.0,0.800000
1,9.0,0.761905,9.0,0.761905,10.0,0.818182,9.0,0.761905,1.0,0.400000,10.0,0.818182
2,10.0,0.588235,10.0,0.500000,11.0,0.666667,10.0,0.588235,2.0,0.400000,10.0,0.588235
3,6.0,0.727273,6.0,0.727273,6.0,0.727273,5.0,0.600000,4.0,0.250000,5.0,0.600000
4,13.0,0.789474,13.0,0.789474,13.0,0.789474,13.0,0.789474,4.0,0.666667,13.0,0.789474


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(df_X_train)
X_test_scaled = scaler.transform(df_X_test)
X_train_scaled

array([[ 0.1854014 , -0.29878695,  0.06185393, ..., -0.11665755,
         0.16169462, -0.40050658],
       [-0.26212613, -0.4160811 , -0.34878997, ...,  0.22791104,
        -0.27990169, -0.5245446 ],
       [ 0.1854014 ,  0.22065857,  0.06185393, ...,  0.12262619,
         0.16169462,  0.14880467],
       ...,
       [-1.38094494, -2.64466996, -1.37539972, ..., -0.40379804,
        -1.38389245, -2.88126705],
       [ 0.1854014 , -0.54717456,  0.26717588, ..., -0.0879435 ,
         0.16169462, -0.66317533],
       [-0.48588989,  1.0696448 , -0.55411192, ..., -0.40379804,
        -0.50069984,  1.0466037 ]])

In [13]:
what_changed = "Implemented Keras"
print(training_scores_y)
print(testing_scores_y)

[4.0, 3.75, 2.8, 3.4, 2.4, 1.333, 4.6, 3.8, 4.2, 2.6, 4.4, 4.2, 5.0, 5.0, 5.0, 3.4, 5.0, 4.0, 3.2, 5.0, 4.4, 3.6, 3.6, 0.8, 1.4, 3.2, 3.8, 4.2, 3.0, 3.6, 3.0, 3.0, 3.0, 4.8, 2.5, 2.25, 3.5, 3.5, 3.0, 3.0, 3.667, 1.667, 3.6, 3.8, 3.4, 3.4, 3.8, 2.8, 3.75, 4.4, 3.75, 2.75, 3.75, 2.25, 3.8, 3.2, 3.8, 4.8, 4.0, 1.4, 4.2, 1.8, 4.2, 2.0, 2.6, 3.25, 5.0, 3.0, 3.75, 4.25, 4.75, 3.25, 3.75, 3.25, 3.8, 5.0, 3.6, 3.8, 2.0, 3.4, 4.2, 3.4, 3.0, 3.0, 3.25, 2.75, 2.6, 3.4, 3.6, 3.6, 4.8, 4.0, 3.6, 3.4, 2.4, 4.222, 3.0, 3.0, 2.75, 4.0, 1.75, 1.75, 4.5, 3.25, 5.0, 3.0, 2.8, 2.667, 4.0, 4.4, 4.25, 4.176, 3.0, 2.5, 3.8, 4.6, 1.4, 2.8, 2.8, 4.2, 1.0, 4.2, 2.75, 2.5, 3.5, 4.75, 3.0, 3.5, 4.0, 2.0, 3.2, 3.6, 3.8, 5.0, 4.0, 3.0, 2.5, 3.0, 4.0, 2.0, 3.0, 2.625, 3.2, 4.0, 3.2, 2.4, 4.4, 3.6, 2.4, 3.6, 4.6, 4.4, 4.4, 4.6, 4.4, 2.6, 3.6, 4.2, 3.2, 2.0, 2.8, 1.6, 3.4, 4.0, 2.6, 2.8, 3.0, 3.6, 5.0, 3.4, 4.0, 4.0, 3.5, 4.5, 3.2, 3.8, 3.2, 4.0, 3.25, 3.0, 4.25, 1.5, 4.25, 4.25, 0.75, 3.0, 3.5, 3.75, 4.0, 1.75, 1.75,

In [14]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
# X_train_scaled
# X_test_scaled
# training_scores_y
# testing_scores_y

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(training_scores_y)
encoded_Y = encoder.transform(training_scores_y)

# baseline
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X_train_scaled, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/jan/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/jan/.local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Error when checking target: expected dense_3 to have shape (10,) but got array with shape (1,)

In [ ]:
# dropout in the input layer with weight constraint
def create_model():
    # create model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
 
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X_train_scaled, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
from sklearn.neural_network import MLPRegressor
itera = 1000
r = MLPRegressor(max_iter=itera)
r.fit(X_train_scaled, training_scores_y)
r.score(X_train_scaled, training_scores_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(X_train_scaled).tolist()
test_prediction = r.predict(X_test_scaled).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
a = pearsonr(training_scores_y, train_prediction)[0]
b = pearsonr(testing_scores_y, test_prediction)[0]
print('MLP Training Accuracy: ',round(a,3))
print('MLP Testing Accuracy: ', round(b,3))
print('MLP Drop Train-Test: ', round(a-b,3))

In [ ]:
print('MLP Training Accuracy:',round(a,3), 'w/mod_:', round(c,3), '| SVR Training Accuracy:',round(e,3),'| KNN Training Accuracy:',round(g,3))
print('MLP Testing Accuracy :', round(b,3), 'w/mod_:', round(d,3),'| SVR Testing Accuracy :', round(f,3), '| KNN Testing Accuracy :', round(h,3))
print('MLP Drop Train-Test  :', round(a-b,3), 'w/mod_:', round(c-d,3),'| SVR Drop Train-Test : ', round(e-f,3), '| KNN Drop Train-Test  :', round(g-h,3))

In [ ]:
#these will be logged to your sklearn-demos project on Comet.ml
params={"main experiment changes":what_changed,
        "random_state":random_state,
        "MLP_iterations": itera,
        "MLP_solver":solv,
        "MLP_activation":activ,
        "SVM_gamma":gamma_type,
        "SVM_C":c_val,
        "SVM_epsilon":epsilon_val
        #"stratify":True
}

metrics = {'MLP Training Accuracy':a,
'MLP Testing Accuracy':b,
'MLP Drop Train-Test':a-b,
'MLP_mod Training Accuracy':c,
'MLP_mod Testing Accuracy':d,
'MLP_mod Drop Train-Test':c-d,
'SVM Training Accuracy':e,
'SVM Testing Accuracy':f,
'SVM Drop Train-Test':e-f,
'KNN Training Accuracy':g,
'KNN Testing Accuracy':h,
'KNN Drop Train-Test':g-h
}

experiment.log_dataset_hash(X_train_scaled)
experiment.log_parameters(params)
experiment.log_metrics(metrics)